In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

In [3]:
from cobra.io import load_json_model

ijomc  = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc.json')
ijofit = load_json_model('/home/laurence/ME/data/dynamicME/best_ijomc.json')

In [4]:
from dynamicme.optimize import Optimizer

In [5]:
opt = Optimizer(ijomc)

In [6]:
gap = opt.add_duality_gap_constraint()

In [9]:
crowding = ijomc.metabolites.crowding
crowdingfit = ijofit.metabolites.crowding
a12_dict = {(gap.metabolites.get_by_id(crowding.id), gap.reactions.get_by_id(rxn.id)):(
    rxn.metabolites[crowding], 
    ijofit.reactions.get_by_id(rxn.id).metabolites[crowdingfit]) for rxn in crowding.reactions}

In [11]:
opt.make_disjunctive_primal_dual(gap, a12_dict)

<Model duality_gap at 0x7f16903ca890>

In [13]:
for rxn in gap.reactions:
    rxn.objective_coefficient = 0.
for rxn in gap.reactions.query('binary_'):
    rxn.objective_coefficient = 1.

In [17]:
ijofit.optimize()
MU_FIT = ijofit.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

In [18]:
MU_FIT

0.7301837182736599

In [21]:
F_TOL = 0.05

gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.upper_bound = MU_FIT*(1+F_TOL)
gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.lower_bound = MU_FIT*(1-F_TOL)

In [22]:
gap.optimize(solver='gurobi', objective_sense='minimize')

<Solution 1.00 at 0x7f168a093c50>

In [23]:
gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

0.7489152420861117

In [24]:
for rxn in gap.reactions.query('binary_'):
    if rxn.x > 0.99:
        print rxn, rxn.x

binary_crowding_ATPS4rpp_abs 1.0


## Plug back in MILP solution to make sure

In [25]:
ijotest  = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc.json')

In [27]:
crowding = ijotest.metabolites.crowding
crowding0 = ijomc.metabolites.crowding
crowding_fit = ijofit.metabolites.crowding

for rxn in ijotest.metabolites.crowding.reactions:        
    rxn0 = ijomc.reactions.get_by_id(rxn.id)
    rxn_fit = ijofit.reactions.get_by_id(rxn.id)
    a1 = rxn0.metabolites[crowding0]
    a2 = rxn_fit.metabolites[crowding_fit]
    binary = gap.reactions.get_by_id('binary_%s_%s'%(crowding0.id, rxn.id))
    if binary > 0.5:
        rxn._metabolites[crowding] = a2
    else:
        rxn._metabolites[crowding] = a1

In [29]:
ijotest.optimize(solver='gurobi')

<Solution 0.73 at 0x7f1689951450>

In [30]:
ijotest.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

0.7260066430633311

In [31]:
ijofit.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

0.7301837182736599

In [32]:
gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

0.7489152420861117